# Нейронные сети. Архитектуры нейронных сетей

## Подготовка данных

В этом практическом заданий мы будем решать задачу классификации цифр на датасете `mnist` с помощью полносвязной и сверточной нейронной сети. Для этого мы будем использовать надстройку над `tensorflow`, которая называется `keras`. Для начала обсудим данные. `mnist` датасет состоит из черно-белых изображений цифр размером $28 \times 28$ пикселей. В данном случае, мы работаем с одним каналом, хотя в случае цветных изображений, общее число каналов равно трем. Загрузим наши данные используя функцию `load_data` объекта `mnist` из модуля `keras.dataset`. Перед выполнением этого задания убедитесь, что ваша версия `tensorflow` >= 1.4.

In [46]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [47]:
x_train.shape

(60000, 28, 28)

Нормализуйте заруженные данные `x_train` и `x_test`. Для этого следует разделить числовое значение каждого пикселя на $255$. Далее, переведите `y_train` и `y_test` в one-hot представление, используя функцию `tf.keras.utils.to_categorical`. Наше первое задание будет заключатся в реализации полносвязной нейронной сети. Поэтому измените размерность тренировочных и тестовых данных с помощью метода `reshape`.
    >> np_vector.shape
    >> (28, 28)
    >> np_vector = np_vector.reshape(28 * 28)
    >> np_vector.shape
    >> (784,)

### *РЕШЕНИЕ*

In [48]:
import numpy as np

In [49]:
def normalize(x, max_value=255):
    return x / max_value

In [50]:
x_train = normalize(x_train)
x_test = normalize(x_test)

In [51]:
# Разворачиваем фичи в одномерный вектор
x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)

In [52]:
num_classes = 10

In [53]:
# Конвертируем лейблы типа 7 в векторы [0,0,0,0,0,0,0,1,0,0]
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## Полносвязная нейронная сеть.

В этой части задания вам предлагается реализовать обычную нейронную сеть с использованием последовательной модели `tf.keras.models.Sequential`. Данная модель позволяет добавлять слои при помощи функции встроенной `add`. Наша нейронная сеть будет состоять всего лишь из одного скрытого слоя с количеством нейроннов равным $256$, функцией активации ReLU и с `input_shape=(784,)`, что соответствует количеству нейронов во входном слое нашей нейронной сети. Количество нейроннов в выходном слое равно количеству классов, в качестве функции активации нужно использовать softmax. Не забудьте вызвать `model.compile` после добавления слоев. Используйте в качестве функции потерь `categorical_crossentropy`, оптимизатор `adadelta` и метрику `accuracy`.

### *РЕШЕНИЕ*

In [54]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [55]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adadelta(),
    metrics=['accuracy']
)

После этого, создайте модель и загрузите веса нейронной сети из файла `neural_networks.h5`. Какое количество настраиваемых параметров содержится в этой нейронной сети. Запишите это число в качестве первого ответа `answer1` на это задание. Оцените качетво на тестовой выборке и запишите это значение с точностью до трех знаков после запятой в переменную `answer2`.

### *РЕШЕНИЕ*

In [56]:
model.load_weights('neural_networks.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________


In [57]:
answer1 = 203530

In [58]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.0660775844285672
Test accuracy: 0.982


In [59]:
answer2 = accuracy

## Сверточная нейронная сеть

Далее, вам предлагается реализовать сверточную нейронную сеть. 

* Размерность входного слоя $(28, 28, 1)$.
* Сверточный слой с $32$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Сверточный слой с $64$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Понижение размерности признаков.
* Полносвязный слой с 64 нейронами
* Выходной слой с количеством нейронов равному количеству классов.

Для этого предлагается использовать следующие классы `Convolution2D`, `MaxPooling2D` и `Flatten` для понижения размерности признаков. Все эти классы как и слой полносвязной нейронной сети `Dense` находятся в `tf.keras.layers`. Используйте ReLU в качестве функции активации во всех слоях, где это потребуется, кроме выходного, в нем по аналогии с прошлым заданием используется softmax. Аналогичная ситуация с функцией `compile` после добавления слоев.

### *РЕШЕНИЕ*

In [60]:
CNN_model = tf.keras.models.Sequential()
CNN_model.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
CNN_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu'))
CNN_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model.add(tf.keras.layers.Flatten())
CNN_model.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [61]:
CNN_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adadelta(),
    metrics=['accuracy']
)

Теперь оцените качество получившейся модели на тестовой выборке. Для этого измените размерность `x_train` и `x_test` на размерность входного слоя. Загрузите веса `conv_net.h5`. Запишите количество параметров этой сверточной нейронной сети в `answer3`. Сравните его с количеством параметром в полносвязной нейронной сети, которую мы реализовали ранее. Оценку качества запишите в `answer4` с точностью до 3 трех знаков после запятой.

### *РЕШЕНИЕ*

In [66]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [67]:
CNN_model.load_weights('conv_net.h5')
CNN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                102464    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                650       
Total para

In [68]:
answer3 = 121930

In [70]:
loss, accuracy = CNN_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.030455502439221346
Test accuracy: 0.9929


In [71]:
answer4 = accuracy

# Строка с ответами

In [72]:
output = "nn params {0}\n nn score {1:.3f}\ncnn params {2}\ncnn score {3:.3f}"
print(output.format(answer1, answer2, answer3, answer4))

nn params 203530
 nn score 0.982
cnn params 121930
cnn score 0.993
